# Reinforcement Learning: Theory and Algorithms
* 第1章における各定理，補題，系などをまとめます．証明は初見でもこれを参考にすれば導出できるように丁寧に記します．
* 必要に応じて実装を行います．
* Occupancy Measureは訪問頻度と訳すことにします．
# LPと強化学習
価値反復法や方策反復法といった動的計画法に基づく手法では，問題の構造や割引率の値などにより，収束までに非常に多くの反復回数を要することがあります．こうした計算の複雑さを統一的に評価するためには，計算時間を入力サイズ，すなわち入力のビット長に対する関数として捉えてみましょう．反復回数は，計算時間の上界として計算理論的に評価されます．MDPの入力には，状態遷移確率$P$，報酬関数 $r$，割引率 $\gamma$ が含まれ，これらはいずれも有限ビット長でコンピュータ上に表現されると仮定します．

では，価値反復を例にとって入力サイズに対する計算時間が多くなる例を考えてみましょう．価値反復における計算量は以下の通りでした．

$$
|\mathcal{S}|^2|\mathcal{A}| \frac{L(P, r, \gamma) \log \frac{1}{1-\gamma}}{1-\gamma}
$$
$P$や$r$は$L$にのみ依存しており，この2つの入力が大きくなっても，計算量としては$L$の多項式的($x^n$倍など)な増加のみです．一方で $\gamma$ を見てみると$L$だけでなく$\frac{1}{1-\gamma}$や$\log \frac{1}{1-\gamma}$に依存しています．$\gamma$の入力ビットは例えば$0.9999$のような値でさえも$O\left(\log \frac{1}{1-\gamma}\right)$ビットで表現できます．つまり小数点以下の数字がどれだけ大きくても，そこまで入力ビットは大きくなりません．一方で，$\gamma$を限りなく1に近づける($0.99999...$)と，価値反復の計算量は爆発的に膨れ上がります．例えば$\gamma=0.9$の場合の価値反復の計算量は$10$ほどですが，$\gamma=0.9999$になると$10000$になります．

つまり，入力ビットはそこまで増えないのに計算量が滅茶苦茶増えます．この問題に対応するために線形計画問題を考えます．ちなみに，線形計画問題における計算量は以下のようになります．

$$
|\mathcal{S}|^3|\mathcal{A}| L(P, r, \gamma)
$$
これは，MDPの入力ビットに対して真の多項式時間のアルゴリズムといえますね．

## 主問題
以下の問題を最小化します．最適化させる変数は$V \in \mathbb{R}^{|\mathcal{S}|}$です．
$$
\sum_{s \in \mathcal{S}} \mu(s) V(s)
$$
目的関数は価値関数の期待値を示しています．つまり，
$$
\sum_s \mu(s) V(s)=\mathbb{E}_{s \sim \mu}[V(s)]
$$
と表現できます．

制約関数は
$$
V(s) \geq r(s, a)+\gamma \sum_{s^{\prime}} P\left(s^{\prime} \mid s, a\right) V\left(s^{\prime}\right) \quad \forall s \in \mathcal{S}, a \in \mathcal{A}
$$
と表現されます．これはベルマン不等式であり右辺がどのような$a$をとっても左辺の$V$が同じかそれ以上であるということを示しています．つまり，以下のベルマン最適方程式と同義であるといえます．

$$
V^*(s)=\max _{a \in \mathcal{A}}\left[r(s, a)+\gamma \sum_{s^{\prime}} P\left(s^{\prime} \mid s, a\right) V^*\left(s^{\prime}\right)\right] \quad \forall s \in \mathcal{S}
$$
この主問題をまとめて書くと以下のようになります．

$$
\begin{array}{cl}
\operatorname{minimize} & \sum_{s \in \mathcal{S}} \mu(s) V(s) \\
\text { subject to } & V(s) \geq r(s, a)+\gamma \sum_{s^{\prime}} P\left(s^{\prime} \mid s, a\right) V\left(s^{\prime}\right) \quad \forall s \in \mathcal{S}, a \in \mathcal{A}
\end{array}
$$
一方でこの問題に対して双対問題を考えましょう．
## ラグランジュ定式化

主問題に対する双対問題は，ラグランジュ双対で導出します．まず，双対変数 $d(s,a) \geq 0$ を，主の不等式制約

$$
V(s) \geq r(s, a) + \gamma \sum_{s'} P(s' \mid s, a) V(s')
$$

に対応させ，ラグランジアンを定義します：

$$
\mathcal{L}(V, d) = \sum_{s} \mu(s) V(s) + \sum_{s,a} d(s,a) \left[ r(s,a) + \gamma \sum_{s'} P(s' \mid s,a)V(s') - V(s) \right]
$$

このラグランジアンを $V$ に関して最小化することでラグランジュ双対関数を得ます：

$$
g(d) = \inf_{V} \mathcal{L}(V, d)
$$

ここで，$V$ に関する項を整理すると：

$$
\mathcal{L}(V, d) = \sum_s \left[ \mu(s) - \sum_a d(s,a) \right] V(s) + \sum_{s,a} d(s,a) \left[ r(s,a) + \gamma \sum_{s'} P(s' \mid s,a) V(s') \right]
$$

このうち，$\sum_{s,a} d(s,a) \cdot \gamma \sum_{s'} P(s' \mid s,a) V(s')$ を書き換えると：

$$
\sum_{s'} V(s') \cdot \gamma \sum_{s,a} d(s,a) P(s' \mid s,a)
$$

よって，ラグランジアンの $V(s)$ に関する係数は，

$$
\mu(s) - \sum_a d(s,a) + \gamma \sum_{s'', a''} d(s'', a'') P(s \mid s'', a'')
$$

となります．これが 0 でないと，$V(s)$ を $\pm \infty$ に飛ばすことで $\inf_V \mathcal{L}(V,d)$ を無限小にできてしまいます．したがって，双対関数が有界になるための条件は：

$$
\sum_a d(s,a) = \mu(s) + \gamma \sum_{s',a'} P(s \mid s',a') d(s',a') \quad \forall s \in \mathcal{S}
$$

この条件は，flow conservation constraintと呼ばれます．これは，状態 $s$ への割引訪問確率の合計が，初期分布と他の状態からの遷移によって決まることを意味します．

## ラグランジュ双対問題の定式化

ラグランジュ双対問題は以下の線形計画問題として書けます：

$$
\begin{array}{cl}
\operatorname{maximize} & \sum_{s,a} d(s,a) r(s,a) \\
\text{subject to} & \sum_a d(s,a) = \mu(s) + \gamma \sum_{s',a'} P(s \mid s',a') d(s',a') \quad \forall s \in \mathcal{S} \\
& d(s,a) \geq 0 \quad \forall (s,a) \in \mathcal{S} \times \mathcal{A}
\end{array}
$$

ここで，$d(s,a)$ は状態行動ペア $(s,a)$ における割引訪問頻度に対応します．この双対問題は，価値関数を変数とした主問題と等価な最適値を持ち，双対ギャップは0です．これはMDPの解を別の側面から捉えたものです．

双対問題の変数 $d(s,a)$ は，ある方策 $\pi$ の下での割引訪問頻度：

$$
d^\pi(s,a) = \sum_{t=0}^{\infty} \gamma^t \mathbb{P}(s_t = s, a_t = a \mid \pi, s_0 \sim \mu)
$$

と解釈できます．このとき，双対問題の可行領域は，すべての方策 $\pi$ に対する $d^\pi$ の集合（＝状態行動ポリトープ）になります．このポリトープは，状態と行動の個数に応じて構成される凸集合です．

## なぜ双対ギャップが0なのか？

線形計画問題において，双対ギャップがゼロになるのは，強双対性（strong duality）が成り立つためです．Slaterの制約条件がそれを成立させます．

### Slater制約条件

Slaterの定理によれば，次の2つ条件が成り立てば，強双対性が成立します：

*   **目的関数が凸であること**
*   **制約条件において厳密可行点が存在すること**

最初の条件は明らかに成り立ちます．なぜなら主問題において目的関数は線形ですので凸関数だからです．
### 厳密可行点の存在

すべての不等式制約を厳密に満たすような $V$ が存在すれば Slater 条件が成立します．具体的には：

$$
V(s) > r(s,a) + \gamma \sum_{s'} P(s' \mid s,a) V(s') \quad \forall (s,a)
$$
を満たす $V$ が存在すればよいです．このような $V$ は以下のように構成できます：

$$
V(s) := \frac{R_{\max}}{1 - \gamma} + \epsilon
$$

ここで，$R_{\max}$ は報酬関数の最大値，$\epsilon > 0$ は任意の正の定数です．このように定数関数として大きな値をとれば，右辺の割引付き将来価値よりも常に大きくなり，厳密可行点の条件を満たします．

よって制約条件が満たされたのでKKT条件が必要十分条件となり，双対ギャップがゼロであることがいえました．
